<a href="https://colab.research.google.com/github/potashov/Practical-Deep-Learning-Book/blob/master/Epoxy_Supervisor_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [2]:
%pip install ultralytics albumentations
import ultralytics
ultralytics.checks()

Ultralytics 8.3.51 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.7/107.7 GB disk)


# Git Clone?

In [6]:
!git clone https://github.com/evbevz/epoxy-supervisor.git

Cloning into 'epoxy-supervisor'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 31 (delta 5), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (31/31), 10.44 KiB | 10.44 MiB/s, done.
Resolving deltas: 100% (5/5), done.


# Predict

In [7]:
# Run inference on an image with YOLO11n
!yolo predict model=yolo11n-pose.pt source='https://ultralytics.com/images/zidane.jpg'

100% 5.97M/5.97M [00:00<00:00, 106MB/s]
Ultralytics 8.3.51 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLO11n-pose summary (fused): 257 layers, 2,866,468 parameters, 0 gradients, 7.4 GFLOPs

100% 49.2k/49.2k [00:00<00:00, 5.85MB/s]
image 1/1 /content/zidane.jpg: 384x640 2 persons, 189.2ms
Speed: 15.8ms preprocess, 189.2ms inference, 31.3ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/pose/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


# Validate default model

In [8]:
# Download COCO val
import torch
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')  # download (780M - 5000 images)
!unzip -q tmp.zip -d datasets && rm tmp.zip  # unzip

100%|██████████| 780M/780M [00:09<00:00, 89.5MB/s]


In [11]:
# Validate YOLO11n on COCO8 val
!yolo val model=yolo11n-pose.pt data=coco8-pose.yaml

Ultralytics 8.3.51 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLO11n-pose summary (fused): 257 layers, 2,866,468 parameters, 0 gradients, 7.4 GFLOPs

Dataset 'coco8-pose.yaml' images not found ⚠️, missing path '/content/datasets/coco8-pose/images/val'
100% 334k/334k [00:00<00:00, 8.42MB/s]
Unzipping /content/datasets/coco8-pose.zip to /content/datasets/coco8-pose...: 100% 27/27 [00:00<00:00, 4290.93file/s]
Dataset download success ✅ (0.9s), saved to /content/datasets

val: Scanning /content/datasets/coco8-pose/labels/val... 4 images, 0 backgrounds, 0 corrupt: 100% 4/4 [00:00<00:00, 74.31it/s]
val: New cache created: /content/datasets/coco8-pose/labels/val.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100% 1/1 [00:01<00:00,  1.04s/it]
                   all          4         14      0.811      0.929      0.917      0.728          1      0.697      0.772      0.355
Speed: 6.9m

# Load Dataset

In [3]:
# Download Epoxy Level Dataset
import torch
torch.hub.download_url_to_file('http://test.bevz.space/robot/dataset/epoxy-level-1774.zip', 'tmp.zip')  # download (3.8G - 1774 images)
!unzip -q tmp.zip -d datasets && rm tmp.zip  # unzip
!touch /content/datasets/.gitignore

100%|██████████| 3.76G/3.76G [06:20<00:00, 10.6MB/s]


# Train

In [ ]:
# Train YOLO11n on COCO8 for 3 epochs
!yolo train model=yolo11n-pose.pt data=datasets/epoxy-level-1774/data.yaml epochs=3 imgsz=640

In [ ]:
# Python Usage
from ultralytics import YOLO

# Load a model
model = YOLO('yolo11n-pose.yaml')  # build a new model from scratch
model = YOLO('yolo11n-pose.pt')  # load a pretrained model (recommended for training)

# Use the model
results = model.train(data='coco8.yaml', epochs=3)  # train the model
results = model.val()  # evaluate model performance on the validation set
results = model('https://ultralytics.com/images/bus.jpg')  # predict on an image
results = model.export(format='onnx')  # export the model to ONNX format